In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [8]:

#載入數據集，one_hot是把数据转化为只有0和1的形式
mnist = input_data.read_data_sets("MNIST_data", one_hot = True) #這步有時候會失效

#因為數據集很大，故我們要用stochastic gradient descent，
#會將資料集分批次（一次100张）放入神经网络进行训练，
#並不會一次將所有資料拿來train (計算量很大)
#每一個批次的大小
batch_size = 100 

#計算一共有多少批次，训练集数量mnist.train.num_examples 
# // 在python中表示取商
n_batch = mnist.train.num_examples // batch_size      

#定义兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
x = tf.placeholder(tf.float32, [None, 784]) # 28 * 28 = 784，None值变为100
y = tf.placeholder(tf.float32, [None, 10]) #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量
keep_prob = tf.placeholder(tf.float32)

#创建一個簡單的神經網路 (只有输入层和輸出層，输入层784个神经元，输出层總共10個神經元，即十个标签)
# 新的初始化方法，stddev标准差
#隐藏层1（2000个神经元）
W1 = tf.Variable(tf.truncated_normal([784, 2000], stddev=0.1))
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob)    #tensorflow封装好的dropout函数

#隐藏层2（2000个神经元）
W2 = tf.Variable(tf.truncated_normal([2000, 2000], stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

#隐藏层3 （1000个神经元）
W3 = tf.Variable(tf.truncated_normal([2000, 1000], stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop, W3) + b3)
L3_drop = tf.nn.dropout(L3, keep_prob)


#输出层4 （10个神经元）
#（实际上并不需要定义这么多神经元，此处只是为了模拟过拟合现象）
#过拟合现象出现的原因有二：一是网络太复杂，二是数据量太小
#网络模型有很多的参数，不好确定，未知数太多，已知的公式太少，求不出应有的解
W4 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
#L4 = tf.nn.tanh(tf.matmul(L3_drop, W4) + b4)
#L4_dropout = tf.nn.dropout(L4, keep_prob)
prediction = tf.nn.softmax(tf.matmul(L3_drop, W4) + b4) 

#W1 = tf.Variable(tf.zeros([784, 10]))              #权值
    # b = tf.Variable(tf.zeros([1, 10]))                #偏置值
#b1 = tf.Variable(tf.zeros([10]))  
#prediction = tf.nn.softmax(tf.matmul(x, W) + b)   #预测值，用到softmax


#二次代價函數 : loss = mean((y - prediction)^2)
#loss = tf.reduce_mean(tf.square(y - prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction)) #交叉熵

#使用梯度下降法
#Gradient desent method  (learning rate = 0.2)
gd = tf.train.GradientDescentOptimizer(0.2)

#最小化 代價函數 (operator)
train_step = gd.minimize(loss)
#以上两句可以合并为 train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量 operator
init = tf.global_variables_initializer()



#测试训练的准确率，求准确率的方法
#如果y標籤最大的值，與prediction標籤最大的值相等，則回傳true
#結果存在一個 boolean 的变量correct_prediction中

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
                              
 #argmax 返回一維張量中最大的值所在的位置
 # 求标签y里面最大的值在哪个位置即标签
 # tf.argmax(prediction, 1)预测 概率最大就会判定识别的这张图片是属于哪个标签的
 # (tf.argmax(y, 1)，真实样本的y存放的都是0或1，哪位是1就会返回哪位的值
 #  然后再比较上面两者，是否一样
                             
                              
# 求准确率
# 轉換資料格式 boolean 轉成 32位的float，接著再取平均值，得到准确率
# true转换为1.0，false转换为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         
                              
#開始training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31): #总共疊代21次（周期） (outer loop)，把所有的图片训练21次
        #每一次 outer loop 不一次拿所有的數據集，來做 Gradient desent，這就是 stochastic gradient descent
        for batch in range(n_batch):#每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs图片数据, batch_ys图片标签
            # mnist.train.next_batch(batch_size)是获取下一个一百张图片               
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
              
           # keep_prob:1.0意思是所有的神经元都工作，此时dropout是没有用的    
            feed_dict = {x: batch_xs, y: batch_ys, keep_prob:1.0}  #拿來feed 的 dictionary                  
            sess.run(train_step, feed_dict)
                              
        #每做完一次 outer loop 計算一次准确率
        #测试数据计算出的准确率
        outer_loop_feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0} #testing data feed dictionary
        test_acc = sess.run(accuracy, outer_loop_feed_dict)
          
         #训练集做测试算出的准确率   
        outer_loop_feed_dict1 = {x: mnist.train.images, y: mnist.train.labels, keep_prob:1.0} #testing data feed dictionary
        train_acc = sess.run(accuracy, outer_loop_feed_dict1)
        #测试图片训练，测试图片测试，得到的一般是好的准确率
        
        # str(epoch)周期数
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(test_acc)+",Training Accuracy=" + str(train_acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter =0, Testing Accuracy =0.9469,Training Accuracy=0.95894545
Iter =1, Testing Accuracy =0.9588,Training Accuracy=0.97554547
Iter =2, Testing Accuracy =0.9618,Training Accuracy=0.9826364
Iter =3, Testing Accuracy =0.9642,Training Accuracy=0.9862546
Iter =4, Testing Accuracy =0.9669,Training Accuracy=0.98816365
Iter =5, Testing Accuracy =0.9684,Training Accuracy=0.9896727
Iter =6, Testing Accuracy =0.9689,Training Accuracy=0.9906727
Iter =7, Testing Accuracy =0.9693,Training Accuracy=0.9913091
Iter =8, Testing Accuracy =0.9701,Training Accuracy=0.99185455
Iter =9, Testing Accuracy =0.9704,Training Accuracy=0.9924545
Iter =10, Testing Accuracy =0.9691,Training Accuracy=0.99285454
Iter =11, Testing Accuracy =0.9706,Training Accuracy=0.9932182
Iter =12, Testing Accuracy =0.9711,Training Accuracy=0.